# Практическое задание № 1. Классификация


**Работу выполнила:**

Алексеева Влада Вадимовна, ITMO ID 367801

# Распознавание активности человека (Human Activity Recognition, HAR)

Необходимо по данным с мобильных сенсоров при помощи прикладных алгоритмов машинного обучения предсказать активность человека по шести классам движений:
* Движется по прямой
* Движется вверх (например, движение по лестнице вверх)
* Движется вниз (например, движение по лестнице вниз)
* Сидит
* Стоит
* Лежит

## Сведения о наборе данных
Набор данных содержит записи датчиков со смартфонов (акселерометр и гироскоп с частотой дискретизации 50 Гц) от 30 участников, выполняющих следующие действия: ходьба, ходьба по лестнице, ходьба по лестнице, сидение, стояние и лежание. Данные были предварительно обработаны при помощи фильтров шума. Набор данных представлен Хорхе Л. Рейес-Ортисом.

Признаки были извлечены из 3-х осевых необработанных сигналов акселерометра и гироскопа tAcc-XYZ и tGyro-XYZ. Эти сигналы были сняты с постоянной частотой 50 Гц. Затем были отфильтрованы с помощью медианного фильтра и низкочастотного фильтра Баттерворта 3-го порядка с частотой 20 Гц для удаления шумов. Аналогичным образом сигнал ускорения был разделен на сигналы ускорения тела и гравитации (`tBodyAcc-XYZ` и `tGravityAcc-XYZ`) с помощью другого низкочастотного фильтра Баттерворта с угловой частотой 0,3 Гц. Линейное ускорение тела и угловая скорость были использованы для получения сигналов "рывка" — (`tBodyAccJerk-XYZ` и `tBodyGyroJerk-XYZ`). Также величина этих трехмерных сигналов была рассчитана с использованием евклидовой нормы — (`tBodyAccMag`, `tGravityAccMag`, `tBodyAccJerkMag`, `tBodyGyroMag`, `tBodyGyroJerkMag`).

Наконец, к некоторым из этих сигналов было применено быстрое преобразование Фурье (БПФ), в результате чего получились `fBodyAcc-XYZ`, `fBodyAccJerk-XYZ`, `fBodyGyro-XYZ`, `fBodyAccJerkMag`, `fBodyGyroMag`, `fBodyGyroJerkMag`.

Набор переменных, которые были оценены по этим сигналам, следующий:
* mean(): Среднее значение
* std(): Стандартное отклонение
* mad(): Среднее абсолютное отклонение
* max(): Наибольшее значение в массиве
* min(): Наименьшее значение в массиве
* sma(): Область величины сигнала
* energy(): Мера энергии. Сумма квадратов, деленная на количество значений.
* iqr(): Интерквартильный размах
* entropy(): Энтропия сигнала
* arCoeff(): Коэффициенты авторегрессии с порядком Burg, равным 4
* correlation(): коэффициент корреляции между двумя сигналами
* maxInds(): индекс частотной составляющей с наибольшей величиной
* meanFreq(): средневзвешенное значение частотных компонент для получения средней частоты
* skewness(): перекос сигнала в частотной области
* kurtosis(): эксцесс сигнала в частотной области
* bandsEnergy(): Энергия частотного интервала в пределах 64 бинов БПФ каждого окна.
* angle(): Угол между векторами.
## Импорт библиотек
Первым делом импортируем необходимые библиотеки для работы с данными:

In [13]:
import os
import numpy as np
import pandas as pd

## Считываем набор данных

В рамках работы воспользуемся предобработанными данными.

In [14]:
def read_data(filename):
    return pd.read_csv(filename)

# Считывание CSV-файла и возвращение его как объект pandas.DataFrame
df = read_data('train.csv')
df.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,1,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,1,STANDING


Вы видите таблицу, где каждая строка – это один сэмпл (окно данных), а столбцы – это 561 извлеченный признак (среднее, стандартное отклонение и т.д. для различных сигналов) плюс два служебных столбца в конце: subject (номер волонтера) и Activity (метка класса, например, 'STANDING').

Теперь, сохраним полный набор данных под следующими четырьмя переменными:

* `train_X`: признаки, используемые для обучения модели
* `train_y`: метки, используемые для обучения модели
* `test_X`: признаки, используемые для проверки модели
* `test_y`: метки, используемые для проверки модели

In [15]:
# Функция, которая загружает и разделяет данные
def load_dataset(label_dict):
    # Матрица признаков для обучения 
    train_X = read_data('train.csv').values[:,:-2] # .values[:,:-2] означает взять все строки и все столбцы, кроме последних двух (последние два – это subject и Activity)
    
    # Вектор целевых меток для обучения 
    train_y = read_data('train.csv')['Activity'] # Сначала выбирается столбец 'Activity'
    train_y = train_y.map(label_dict).values # .map(label_dict) текстовые метки заменяются на числа из словаря
    # .values преобразует результат в массив numpy

    # Аналогичным образом загружаются данные из файла test.csv для финальной оценки модели
    test_X = read_data('test.csv').values[:,:-2]
    test_y = read_data('test.csv')
    test_y = test_y['Activity'].map(label_dict).values

    return(train_X, train_y, test_X, test_y)

label_dict = {'WALKING':0, 'WALKING_UPSTAIRS':1, 'WALKING_DOWNSTAIRS':2, 'SITTING':3, 'STANDING':4, 'LAYING':5}

train_X, train_y, test_X, test_y = load_dataset(label_dict)

## Выбор моделей для сравнения

На основе анализа литературы, для задачи Human Activity Recognition (HAR) часто используются и показывают хорошие результаты следующие классические модели: **k-ближайших соседей (k-NN), метод опорных векторов (SVM) и случайный лес (Random Forest**). Эти модели были выбраны для сравнения полученных результатов.

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# Словарь, содержащий экземпляры (инициализированные, но не обученные) моделей
# Параметры по умолчанию задаются здесь, но они будут перезаписаны в процессе подбора гиперпараметров
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'k-Nearest Neighbors': KNeighborsClassifier(),
    'Support Vector Machine': SVC(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42)
}

# Словарь, который для каждой модели определяет сетку гиперпараметров для перебора
# Например, для k-Nearest Neighbors будет проверено, какая модель лучше: с 3, 5 или 7 соседями
param_grids = {
    'Logistic Regression': {'C': [0.1, 1, 10]},
    'k-Nearest Neighbors': {'n_neighbors': [3, 5, 7]},
    'Support Vector Machine': {'C': [1, 10, 100], 'gamma': ['scale', 0.1, 0.01], 'kernel': ['rbf']},
    'Random Forest': {'n_estimators': [100, 500, 1000], 'max_depth': [None, 10, 20]}
}

# Мотивация выбора и описание гиперпараметров

**Гиперпараметры** — это конфигурационные настройки, которые определяются до начала процесса обучения модели и не обучаются на данных. Они задаются исследователем и напрямую влияют на структуру модели и процесс ее обучения.

**Logistic Regression** – `C` контролирует компромисс между подгонкой к обучающим данным и простотой модели (регуляризацией). Низкое `C` приводит к более простой, но, возможно, недообученной модели. Высокое `C` позволяет модели лучше подстроиться под данные, но рискует переобучиться.

**k-Nearest Neighbors** – `n_neighbors` определяет, сколько соседей учитывать при голосовании. Маленькое значение делает модель чувствительной к шуму (высокая дисперсия), большое значение сглаживает границы классов и может привести к недообучению (высокое смещение).

**Support Vector Machine (SVM)** – `C` управляет компромиссом между максимизацией ширины разделяющей полосы и минимизацией ошибок классификации. `gamma` (для RBF-ядра) определяет, насколько далеко простирается влияние одного обучающего примера. Низкое `gamma` означает дальнее влияние, высокое `gamma` ближнее влияние.

**Random Forest** – `n_estimators` (число деревьев) обычно ведет к улучшению качества до определенного предела, после чего прирост становится минимальным. `max_depth` ограничивает сложность каждого дерева; слишком большая глубина может привести к переобучению отдельных деревьев, но ансамбль в целом к этому устойчив.

## Обучение, оценка и сравнение моделей

Для объективного сравнения моделей недостаточно использовать однократное разбиение на train/test. Лучше всего использовать **кросс-валидацию** на обучающем наборе, чтобы оценить обобщающую способность каждой модели до финальной оценки на тестовом наборе.

In [17]:
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score

results = []
best_model = {}

# Создается объект для стратифицированной 5-кратной кросс-валидации
# Это означает, что при каждом разбиении данных на 5 частей, пропорции классов в каждой части будут такими же, как и в полном наборе
# Это важно для несбалансированных данных
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Обучение на полном train_X
for name, model in models.items():
    print(f'Обработка: {name}')

    # GridSearchCV
    # Перебирает все комбинации гиперпараметров из param_grids 
    # Для каждой комбинации проводит 5-кратную кросс-валидацию на train_X 
    # Оценивает каждую комбинацию по метрике f1_macro (средневзвешенная F1-мера по всем классам) 
    # Находит комбинацию гиперпараметров, которая дала наилучший результат на кросс-валидации
    grid = GridSearchCV(model, param_grids[name], cv=cv, scoring='f1_macro', n_jobs=-1)

    grid.fit(train_X, train_y) # Запускает процесс подбора гиперпараметров и обучения
    best_model[name] = grid.best_estimator_ # Лучшая модель (с оптимальными гиперпараметрами), найденная в процессе поиска
    cv_score = grid.best_score_ # Лучший результат на кросс-валидации (CV F1-macro)

    # Используем лучшую модель для предсказания на тестовом наборе
    y_pred = grid.best_estimator_.predict(test_X)

    # Вычисляются две финальные метрики качества на тестовых данных
    test_acc = accuracy_score(test_y, y_pred)
    test_f1 = f1_score(test_y, y_pred, average='macro')

    results.append({
        'Model': name,
        'Best Params': grid.best_params_,
        'CV F1-macro': round(cv_score, 4),
        'Test Accuracy': round(test_acc, 4),
        'Test F1-macro': round(test_f1, 4)
    })

Обработка: Logistic Regression
Обработка: k-Nearest Neighbors
Обработка: Support Vector Machine
Обработка: Random Forest


In [18]:
# Визуализация результатов кросс-валидации
results_df = pd.DataFrame(results)
print('\nСравнительная таблица моделей:')
print(results_df.to_string(index=False))


Сравнительная таблица моделей:
                 Model                                Best Params  CV F1-macro  Test Accuracy  Test F1-macro
   Logistic Regression                                   {'C': 1}       0.9872         0.9610         0.9609
   k-Nearest Neighbors                         {'n_neighbors': 3}       0.9708         0.8907         0.8878
Support Vector Machine {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}       0.9919         0.9654         0.9649
         Random Forest     {'max_depth': 20, 'n_estimators': 100}       0.9820         0.9237         0.9220


# Объяснение метрик

- `precision` **(Точность)**: Из всех случаев, которые модель предсказала как определенный класс (например, 'STANDING'), сколько на самом деле были этим классом? Это важно, когда ложные срабатывания (например, сказать, что человек идет вверх, когда он на самом деле стоит) нежелательны.

- `recall` **(Полнота)**: Из всех реальных случаев определенного класса (все истинные 'STANDING'), сколько модель нашла и правильно предсказала? Это важно, когда пропуск события критичен.

- `F1-score`: Гармоническое среднее между precision и recall. Это единая метрика, которая учитывает оба аспекта. F1-score близок к 1, когда обе метрики высоки, и близок к 0, когда хотя бы одна из них низкая.

## Оценка лучшей модели
Используем обученную модель для прогнозирования активности движения, используя признаки из тестового набора (`test_X`). Прогнозы сохраняем в списке `yhat`.

In [19]:
# Определение лучшей модели на основе Test F1-macro
best_name = results_df.loc[results_df['Test F1-macro'].idxmax(), 'Model']
best_model = best_model[best_name]

# Предсказания для лучшей модели
yhat = best_model.predict(test_X)
yhat

array([4, 4, 4, ..., 1, 1, 1], shape=(2947,))

In [20]:
from sklearn.metrics import classification_report
target_names = ['WALKING', 'WALKING UPSTAIRS', 'WALKING DOWNSTAIRS', 'SITTING', 'STANDING', 'LAYING']

print(f'Отчет о классификации для лучшей модели: {best_name}\n')

# Показывает метрики precision, recall и f1-score для каждого класса отдельно, а также средние значения (macro avg и weighted avg)
print(classification_report(test_y, y_pred, target_names=target_names))

Отчет о классификации для лучшей модели: Support Vector Machine

                    precision    recall  f1-score   support

           WALKING       0.89      0.96      0.92       496
  WALKING UPSTAIRS       0.88      0.89      0.89       471
WALKING DOWNSTAIRS       0.97      0.85      0.91       420
           SITTING       0.90      0.90      0.90       491
          STANDING       0.91      0.91      0.91       532
            LAYING       1.00      1.00      1.00       537

          accuracy                           0.92      2947
         macro avg       0.93      0.92      0.92      2947
      weighted avg       0.93      0.92      0.92      2947



## Интерпретация:

- `support`: Число примеров каждого класса в тестовом наборе.

- `precision` **для 'WALKING UPSTAIRS' = 0.88**: Из всех случаев, когда модель сказала 'движение вверх', в 88% случаев она была права.

- `recall` **для 'WALKING UPSTAIRS' = 0.89**: Из всех реальных случаев 'движения вверх' модель правильно нашла 89%.

- `f1-score`: Гармоническое среднее между precision и recall. 

- `accuracy`: Общая точность – 92% из всех предсказаний были верны.

- `macro avg` **(Макро-усреднение)**: Простое арифметическое среднее. Все классы имеют равный вес, независимо от того, сколько в них примеров (их support).

- `weighted avg` **(Взвешенное усреднение)**:  Среднее значение, взвешенное по количеству примеров (support) в каждом классе. Классы с большим количеством примеров влияют на итоговый результат сильнее.